### Способы решения данной задачи:
* Решение в лоб - перебрать тексты regex выржанием по словарю синонимов. Иначе - поиск с помощью ключевых слов категорий
* Использование статистического анализа для извлечения важности слова в тексте
* Классификация с помощью структурных признаков
* Получение списка схожих слов с помощью embeddings, получение коллокаций с помощью этого списка



Теперь загрузим корпус отзывов про товары:

In [ ]:
!wget http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Cell_Phones_and_Accessories_5.json.gz

--2021-12-18 16:17:56--  http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Cell_Phones_and_Accessories_5.json.gz
Resolving snap.stanford.edu (snap.stanford.edu)... 171.64.75.80
Connecting to snap.stanford.edu (snap.stanford.edu)|171.64.75.80|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 45409631 (43M) [application/x-gzip]
Saving to: ‘reviews_Cell_Phones_and_Accessories_5.json.gz.2’

reviews_Cell_Phones 100%[===================>]  43.31M  14.7MB/s    in 2.9s    

2021-12-18 16:18:00 (14.7 MB/s) - ‘reviews_Cell_Phones_and_Accessories_5.json.gz.2’ saved [45409631/45409631]



In [ ]:
import pandas as pd
import gzip

def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

df = getDF('reviews_Cell_Phones_and_Accessories_5.json.gz')

In [ ]:
import nltk
import re

from nltk.collocations import *
from nltk.tokenize import word_tokenize, sent_tokenize

import nltk.corpus
from nltk.corpus import stopwords

nltk.download('punkt')
nltk.download('genesis')
nltk.download('stopwords')

stop = stopwords.words('english')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package genesis to /root/nltk_data...
[nltk_data]   Package genesis is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Загрузим предобученную модель word2vec, найдем похожие слова:

In [ ]:
!wget -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

--2021-12-18 16:18:24--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.38.94
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.38.94|:443... connected.
HTTP request sent, awaiting response... 416 Requested Range Not Satisfiable

    The file is already fully retrieved; nothing to do.



In [ ]:
import gensim
model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz',binary=True,limit=100000)

In [ ]:
# Ищем похожие слова
top=model.most_similar(positive=['iPhone'], topn = 10)
result_search = []
for i in top:
  result_search.append(i[0])
print(result_search)

Зададим функции вывода n-грам:

In [ ]:
def nbest(self, score_fn, n):
    return [p for p,s in self.score_ngrams(score_fn)[:n]]

In [ ]:
def score_ngrams(self, score_fn):
    return sorted(self._score_ngrams(score_fn),
                  key=itemgetter(1), reverse=True)

Найдем n-граммы и отсортируем их по различным метрикам:

In [ ]:


text = df['reviewText'][50]
print(text)
creature_filter = lambda *w: result_search not in w
trigram_measures = nltk.collocations.TrigramAssocMeasures()
bigram_measures = nltk.collocations.BigramAssocMeasures()

text = re.sub(r"(@\[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", text)
text = " ".join([word for word in text.split() if word not in (stop)])

finder_b = BigramCollocationFinder.from_words(word_tokenize(text))
finder_t = TrigramCollocationFinder.from_words(word_tokenize(text))

#finder_b.apply_ngram_filter(creature_filter)
#finder_t.apply_ngram_filter(creature_filter)


In [ ]:
print('Result for trigrams.')
print('likelihood_ratio: ')
for i in finder_t.score_ngrams(trigram_measures.likelihood_ratio):
    print(i)
print('pmi: ')
for i in finder_t.score_ngrams(trigram_measures.pmi):
    print(i)
print('jaccard: ')
for i in finder_t.score_ngrams(trigram_measures.jaccard):
    print(i)

print('Result for bigrams.')
print('likelihood_ratio')
for i in finder_b.score_ngrams(bigram_measures.likelihood_ratio):
    print(i)
print('pmi: ')
for i in finder_b.score_ngrams(bigram_measures.pmi):
    print(i)
print('jaccard: ')
for i in finder_b.score_ngrams(bigram_measures.jaccard):
    print(i)